In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import sklearn.metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
from data_quality import cargue_datasets_c
casos, hurto_tp, mede_victimas, compar = cargue_datasets_c()
hurto_ml = hurto_tp 

from data_quality import casos_c
casos=casos_c(casos)

from data_quality import hurto_tp_c
hurto_tp= hurto_tp_c(hurto_tp)

from data_quality import medevic_c
mede_victimas = medevic_c(mede_victimas)

In [3]:
# Codificar variables categóricas (como Sexo, Estado_civil, Transporte, Modalidad, Conducta, Comuna, Lugar, Bien)
categorical_cols = ['Sexo', 'Estado_civil', 'Transporte', 'Modalidad', 'Conducta', 'Comuna', 'Lugar', 'Bien']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    hurto_ml[col] = le.fit_transform(hurto_ml[col])
    label_encoders[col] = le

In [4]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = hurto_ml.drop(['Fecha','Sexo'], axis=1)  # Elimina las columnas
y = hurto_ml['Sexo']  # Define tu objetivo, en este caso 'Sexo'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
def compare_metrics(learners,X,y,metrics,train_size,repeats):
    strlearners=[]
    for i in range(0,len(learners)):
        strlearners.append(str(learners[i]))  
    df=pd.DataFrame(columns=metrics,index=strlearners)
    for i in range (0,len(learners)):
        for j in range(0,len(metrics)):
            scores=[]
            for k in range(repeats):
                X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=train_size)
                learners[i].fit(X_train,y_train)
                y_hat = learners[i].predict(X_test)
                y_prob = learners[i].predict_proba(X_test)
                if metrics[j]=="Log loss":
                    scores.append(sklearn.metrics.log_loss(y_test, y_prob))
                if metrics[j]=="Brier Score Loss":
                    scores.append(sklearn.metrics.brier_score_loss(y_test, y_prob[:,1]))
                if metrics[j]=="Accuracy":
                    scores.append(sklearn.metrics.accuracy_score(y_test, y_hat))
                if metrics[j]=="Precision":
                    scores.append(sklearn.metrics.precision_score(y_test, y_hat))
                if metrics[j]=="F1":
                    scores.append(sklearn.metrics.f1_score(y_test, y_hat))
                if metrics[j]=="Recall":
                    scores.append(sklearn.metrics.recall_score(y_test, y_hat))
                if metrics[j]=="Auroc":
                    scores.append(sklearn.metrics.roc_auc_score(y_test, y_prob[:,1]))
            df.iloc[i,j]=np.mean(scores)
    return df

In [6]:
metricas=["Log loss","Brier Score Loss","Accuracy","Precision","F1","Recall","Auroc"]
decision_tree=DecisionTreeClassifier()
naive_bayes=GaussianNB()
kNN=KNeighborsClassifier()
log_regression = LogisticRegression()
RandomForest = RandomForestClassifier()
elearn=[decision_tree,naive_bayes,kNN,log_regression,RandomForest]

In [7]:
hurto_ml_scores  = compare_metrics(elearn,X,y,metricas,0.8,20)
hurto_ml_scores

c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting 

,Log loss,Brier Score Loss,Accuracy,Precision,F1,Recall,Auroc
DecisionTreeClassifier(),10.82505,0.313425,0.686713,0.680199,0.667515,0.660664,0.683933
GaussianNB(),0.716316,0.245501,0.592938,0.568352,0.626928,0.701259,0.664085
KNeighborsClassifier(),2.217842,0.248716,0.623709,0.612541,0.620946,0.632752,0.663662
LogisticRegression(),0.653119,0.231351,0.614757,0.606639,0.615545,0.63018,0.662811
RandomForestClassifier(),0.61338,0.18316,0.731746,0.725804,0.724961,0.725516,0.803468
